# Análisis del riesgo de incumplimiento de los prestatarios

Tu proyecto consiste en preparar un informe para la división de préstamos de un banco. Deberás averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

Tu informe se tendrá en cuenta al crear una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

[En este cuaderno se te brindan pistas, breves instrucciones y sugerencias para pensar. No los ignores, ya que están diseñados para equiparte con la estructura del proyecto y te ayudarán a analizar lo que estás haciendo en un nivel más profundo. Antes de enviar tu proyecto, asegúrate de eliminar todas las sugerencias y descripciones que se te hayan proporcionado. Más bien, haz que este informe parezca como si se lo estuvieras enviando a tus compañeros de equipo para demostrar tus hallazgos: ¡no deben saber que recibiste ayuda externa de nuestra parte! Para ayudarte, hemos colocado las pistas que debes eliminar entre corchetes.]

[Antes de sumergirte en el análisis de tus datos, explica los propósitos del proyecto y las hipótesis que vas a evaluar.]

## Abre el archivo de datos y mira la información general. 

[Empieza con la importación de las librerías y la carga de los datos. Es posible que te des cuenta de que necesitas librerías adicionales a medida que avanzas, lo cual es totalmente normal, solo asegúrate de actualizar esta sección cuando lo hagas.]

In [1]:
# Cargar todas las librerías
import pandas as pd
import numpy as np

# Carga los datos
try:
    banco = pd.read_csv('credit_scoring_eng.csv')

except:
    banco = pd.read_csv('/datasets/credit_scoring_eng.csv')

## Ejercicio 1. Exploración de datos

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

[Ahora vamos a explorar nuestros datos. Querrás ver cuántas columnas y filas hay, observa algunas filas para identificar posibles problemas con los datos.]

In [2]:
# Vamos a ver cuántas filas y columnas tiene nuestro conjunto de datos
print(f'El número de filas en le DataSet es de: {banco.shape[0]}')
print(f'El número de columnas en le DataSet es de: {banco.shape[1]}')


El número de filas en le DataSet es de: 21525
El número de columnas en le DataSet es de: 12


In [3]:
# vamos a mostrar las primeras filas N
banco.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


[Describe lo que ves y notas en tu muestra de datos mostrada en la pantalla. ¿Existe algún problema que pueda necesitar investigación y cambios adicionales?]

In [4]:
# Obtener información sobre los datos
banco.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


[¿Hay valores ausentes en todas las columnas o solo en algunas? Describe brevemente lo que ves en 1 o 2 oraciones.]

In [5]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos
banco[banco['days_employed'].isna()]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


[¿Los valores ausentes parecen simétricos? ¿Podemos estar seguros de esta suposición? Explica brevemente tus pensamientos en esta sección. Probablemente desees realizar investigaciones adicionales y contar los valores ausentes en todas las filas con valores ausentes para confirmar que las muestras que faltan son del mismo tamaño.]

In [6]:
days_employed_nan = banco[banco['days_employed'].isna()]
print(f'La cantidad de valores ausentes en days_employed es: {len(days_employed_nan)}')


La cantidad de valores ausentes en days_employed es: 2174


In [7]:
total_income_nan = banco[banco['total_income'].isna()]
print(f'La cantidad de valores ausentes en days_employed es: {len(total_income_nan)}')

La cantidad de valores ausentes en days_employed es: 2174


In [8]:
# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.
days_employed_nan_count = banco[banco['days_employed'].isna()].count()
days_employed_nan_count

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

In [9]:
banco_filtrado = banco[(banco['days_employed'].isna())&(banco['total_income'].isna())]
banco_filtrado.count()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

**Conclusión intermedia**

[¿El número de filas en la tabla filtrada coincide con el número de valores ausentes? ¿Qué conclusión podemos sacar de esto?]

[Calcula el porcentaje de los valores ausentes en comparación con el conjunto de datos completo. ¿Se trata de una porción de datos considerablemente grande? Si es así, es posible que quieras completar los valores ausentes. Para hacer eso, primero debemos definir si los datos ausentes podrían deberse a la característica específica del cliente, como el tipo de empleo u otra cosa. Tendrás que decidir qué característica, según *tú*, podría ser la razón. En segundo lugar, debemos verificar si los valores ausentes dependen de alguna manera del valor de otros indicadores con las columnas con características de clientes, específicas e identificadas.]

[Explica tus próximos pasos y cómo se correlacionan con las conclusiones que has hecho hasta ahora.]

In [10]:
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada y la columna con los valores ausentes
porcentaje_valor_ausente =banco[banco['days_employed'].isna()].shape[0]*100/ banco.shape[0]
print(porcentaje_valor_ausente,'%')


10.099883855981417 %


In [11]:
# Comprobación de la distribución
banco_filtrado = banco[(banco['days_employed'].isna())&(banco['total_income'].isna())]
banco_filtrado.describe()



,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,2174.000000,0.0,2174.000000,2174.000000,2174.000000,2174.000000,0.0
mean,0.552438,NaN,43.632015,0.800828,0.975161,0.078197,NaN
std,1.469356,NaN,12.531481,0.530157,1.418220,0.268543,NaN
min,-1.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN
25%,0.000000,NaN,34.000000,0.250000,0.000000,0.000000,NaN
50%,0.000000,NaN,43.000000,1.000000,0.000000,0.000000,NaN
75%,1.000000,NaN,54.000000,1.000000,1.000000,0.000000,NaN
max,20.000000,NaN,73.000000,3.000000,4.000000,1.000000,NaN


Describe aquí tus hallazgos.]

**Posibles razones por las que hay valores ausentes en los datos**

[Propón tus ideas sobre por qué crees que los valores pueden estar ausentes. ¿Crees que están ausentes al azar o hay algún patrón?]

[Empecemos a comprobar si los valores ausentes son aleatorios.]

In [12]:
# Comprobando la distribución en el conjunto de datos entero
banco.describe()


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


**Conclusión intermedia**

[¿Es similar la distribución en el conjunto de datos original a la distribución de la tabla filtrada? ¿Qué significa eso para nosotros?]

[Si crees que aún no podemos llegar a ninguna conclusión, investiguemos más a fondo nuestro conjunto de datos. Pensemos en otras razones que podrían llevar a la ausencia de datos y verifiquemos si podemos encontrar algún patrón que nos haga pensar que los valores ausentes no son aleatorios. Ya que es tu trabajo, esta sección es opcional.]

In [13]:
# Comprueba otras razones y patrones que podrían llevar a valores ausentes
def comprobacion_relacion_datos(colum):
    days_employed_nan = banco[banco['days_employed'].isna()]
    days_employed = days_employed_nan.groupby(colum)[colum].count()
    return days_employed


In [14]:
comprobacion_relacion_datos('education')

education
BACHELOR'S DEGREE        23
Bachelor's Degree        25
PRIMARY EDUCATION         1
Primary Education         1
SECONDARY EDUCATION      67
SOME COLLEGE              7
Secondary Education      65
Some College              7
bachelor's degree       496
primary education        19
secondary education    1408
some college             55
Name: education, dtype: int64

In [15]:
banco['education'] = list(map(lambda x: x.lower(), banco['education']))


In [16]:
comprobacion_relacion_datos('education')

education
bachelor's degree       544
primary education        21
secondary education    1540
some college             69
Name: education, dtype: int64

In [17]:
comprobacion_relacion_datos('income_type')

income_type
business          508
civil servant     147
employee         1105
entrepreneur        1
retiree           413
Name: income_type, dtype: int64

In [18]:
comprobacion_relacion_datos('family_status')

family_status
civil partnership     442
divorced              112
married              1237
unmarried             288
widow / widower        95
Name: family_status, dtype: int64

**Conclusión intermedia**

[¿Podemos finalmente confirmar que los valores ausentes son accidentales? Verifica cualquier otra cosa que creas que podría ser importante aquí.]

In [19]:
# Comprobación de otros patrones: explica cuáles

**Conclusiones**

[¿Encontraste algunos patrones? ¿Cómo llegaste a esta conclusión?]

[Explica cómo abordarás los valores ausentes. Ten en cuenta las categorías en las que faltan valores.]

[Planifica brevemente tus próximos pasos en la transformación de datos. Probablemente tendrás que abordar diferentes tipos de problemas: duplicados, diferentes registros, artefactos incorrectos y valores ausentes.]

## Transformación de datos

[Repasemos cada columna para ver qué problemas podemos tener en ellas.]

[Comienza con la eliminación de duplicados y la corrección de la información educativa si es necesario.]

In [20]:
# Veamos todos los valores en la columna de educación para verificar si será necesario corregir la ortografía y qué habrá que corregir exactamente


In [21]:
# Arregla los registros si es necesario


In [22]:
# Comprobar todos los valores en la columna para asegurarnos de que los hayamos corregido



[Comprueba los datos de la columna `children`]

In [23]:
# Veamos la distribución de los valores en la columna `children`
banco['children'].unique()


array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [24]:
banco[banco['children'] == 20]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,secondary education,1,married,0,M,business,0,23253.578,purchase of the house
720,20,-855.595512,44,secondary education,1,married,0,F,business,0,18079.798,buy real estate
1074,20,-3310.411598,56,secondary education,1,married,0,F,employee,1,36722.966,getting an education
2510,20,-2714.161249,59,bachelor's degree,0,widow / widower,2,F,employee,0,42315.974,transactions with commercial real estate
2941,20,-2161.591519,0,secondary education,1,married,0,F,employee,0,31958.391,to buy a car
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,-1240.257910,40,secondary education,1,married,0,F,employee,1,21363.842,to own a car
21325,20,-601.174883,37,secondary education,1,married,0,F,business,0,16477.771,profile education
21390,20,NaN,53,secondary education,1,married,0,M,business,0,NaN,buy residential real estate
21404,20,-494.788448,52,secondary education,1,married,0,M,business,0,25060.749,transactions with my real estate


[¿Hay cosas extrañas en esta columna? Si es así, ¿cuál es el porcentaje de datos problemáticos? ¿Cómo es posible que hayan ocurrido? Toma una decisión sobre lo que harás con estos datos y explica tu razonamiento.]

In [25]:
# [arregla los datos según tu decisión]
banco['children'] = banco['children'].replace(20,2)
banco['children'] = banco['children'].replace(-1,1)

In [26]:
# Comprobar la columna `children` de nuevo para asegurarnos de que todo está arreglado
banco['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

[Comprueba los datos en la columna `days_employed`. En primer lugar, piensa qué tipo de problemas podría haber, qué posiblemente desees comprobar y cómo lo harás.]

In [27]:
# Encuentra datos problemáticos en `days_employed`, si existen, y calcula el porcentaje
banco['days_employed'].unique()

array([-8437.67302776, -4024.80375385, -5623.42261023, ...,
       -2113.3468877 , -3112.4817052 , -1984.50758853])

[Si la cantidad de datos problemáticos es alta, podría deberse a problemas técnicos. Puede que queramos proponer la razón más obvia por la que podría haber sucedido y cuáles podrían haber sido los datos correctos, ya que no podemos eliminar estas filas problemáticas.]

In [28]:
# Aborda los valores problemáticos, si existen.
banco['days_employed'] = np.abs(banco['days_employed'])


In [29]:
# Comprueba el resultado - asegúrate de que esté arreglado
banco['days_employed'].unique()


array([8437.67302776, 4024.80375385, 5623.42261023, ..., 2113.3468877 ,
       3112.4817052 , 1984.50758853])

Ahora echemos un vistazo a la edad de clientes para ver si hay algún problema allí. Una vez más, piensa qué datos pueden ser extraños en esta columna, es decir, qué dato no puede ser la edad de alguien.]

In [30]:
# Revisa `dob_years` en busca de valores sospechosos y cuenta el porcentaje
banco['dob_years'].unique()


array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75], dtype=int64)

[Decide qué harás con los valores problemáticos y explica por qué.]

In [31]:
# Resuelve los problemas en la columna `dob_years`, si existen
media = banco['dob_years'].mean()
mediana = banco['dob_years'].median()

print(media)
print(mediana)

banco['dob_years'] = banco['dob_years'].replace(0,mediana)


43.29337979094077
42.0


In [32]:
# Comprueba el resultado - asegúrate de que esté arreglado
banco['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75], dtype=int64)

[Ahora revisemos la columna `family_status`. Mira qué tipo de valores hay y qué problemas puedes tener que abordar.]

In [33]:
# Veamos los valores de la columna

banco['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

In [34]:
# Aborda los valores problemáticos en `family_status`, si existen

banco['family_status'] = banco['family_status'].replace('widow / widower','widow_widower')
banco['family_status'] = banco['family_status'].replace('civil partnership','civil_partnership')

In [35]:
# Comprueba el resultado - asegúrate de que esté arreglado
banco['family_status'].unique()

array(['married', 'civil_partnership', 'widow_widower', 'divorced',
       'unmarried'], dtype=object)

[Ahora revisemos la columna `gender`. Mira qué tipo de valores hay y qué problemas puedes tener que abordar]

In [36]:
# Veamos los valores en la columna
banco['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [37]:
# Aborda los valores problemáticos, si existen

In [38]:
# Comprueba el resultado - asegúrate de que esté arreglado



[Ahora vamos a revisar la columna `income_type`. Mira qué tipo de valores hay y qué problemas puedes tener que abordar]

In [39]:
# Veamos los valores en la columna
banco['income_type'].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

In [40]:
# Aborda los valores problemáticos, si existen
banco['income_type'] = banco['income_type'].replace('paternity / maternity leave','paternity_maternity_leave')
banco['income_type'] = banco['income_type'].replace('civil servant','civil_servant')

In [41]:
# Comprueba el resultado - asegúrate de que esté arreglado
banco['income_type'].unique()


array(['employee', 'retiree', 'business', 'civil_servant', 'unemployed',
       'entrepreneur', 'student', 'paternity_maternity_leave'],
      dtype=object)

[Ahora veamos si hay duplicados en nuestros datos. Si los hay, tendrás que decidir qué harás con ellos y explicar por qué.]

In [42]:
# Comprobar los duplicados
banco[banco.duplicated()]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41,secondary education,1,married,0,F,employee,0,NaN,purchase of the house for my family
3290,0,NaN,58,secondary education,1,civil_partnership,1,F,retiree,0,NaN,to have a wedding
4182,1,NaN,34,bachelor's degree,0,civil_partnership,1,F,employee,0,NaN,wedding ceremony
4851,0,NaN,60,secondary education,1,civil_partnership,1,F,retiree,0,NaN,wedding ceremony
5557,0,NaN,58,secondary education,1,civil_partnership,1,F,retiree,0,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,NaN,64,secondary education,1,married,0,F,retiree,0,NaN,supplementary education
21032,0,NaN,60,secondary education,1,married,0,F,retiree,0,NaN,to become educated
21132,0,NaN,47,secondary education,1,married,0,F,employee,0,NaN,housing renovation
21281,1,NaN,30,bachelor's degree,0,married,0,F,employee,0,NaN,buy commercial real estate


In [43]:
# Aborda los duplicados, si existen
banco = banco.drop_duplicates().reset_index(drop = True)

In [44]:
# Última comprobación para ver si tenemos duplicados
banco.duplicated().sum()

0

In [45]:
# Comprueba el tamaño del conjunto de datos que tienes ahora, después de haber ejecutado estas primeras manipulaciones
banco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21453 entries, 0 to 21452
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21453 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21453 non-null  int64  
 3   education         21453 non-null  object 
 4   education_id      21453 non-null  int64  
 5   family_status     21453 non-null  object 
 6   family_status_id  21453 non-null  int64  
 7   gender            21453 non-null  object 
 8   income_type       21453 non-null  object 
 9   debt              21453 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21453 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


[Describe tu nuevo conjunto de datos: di brevemente qué has cambiado y cuál es el porcentaje de cambios, si hubo alguno.]


# Trabajar con valores ausentes

[Para acelerar el trabajo con algunos datos, puede que necesites trabajar con diccionarios para algunos valores, en los que se proporcionan IDs. Explica por qué y con qué diccionarios vas a trabajar.]

In [46]:
# Encuentra los diccionarios


### Restaurar valores ausentes en `total_income`

[Indica brevemente qué columnas tienen valores ausentes que debes abordar. Explica cómo las arreglarás.]


[Empieza por abordar los valores ausentes del ingreso total. Crea una categoría de edad para los clientes. Crea una nueva columna con la categoría de edad. Esta estrategia puede ayudar a calcular valores para el ingreso total.]


In [47]:
# Vamos a escribir una función que calcule la categoría de edad
def categoria_edad(edades):
    L = []
    for age in edades:
        if age <= 35:
            L.append('adulto_joven')
        elif age <= 60:
            L.append('adulto')
        else:
            L.append('tercera_edad')
    return L
        

In [48]:
# Prueba si la función funciona bien
L =categoria_edad(banco['dob_years'])

In [49]:
# Crear una nueva columna basada en la función
banco['categoria_edad'] = L


In [50]:
# Comprobar cómo los valores en la nueva columna
banco.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,categoria_edad
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adulto
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adulto
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adulto_joven
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adulto_joven
4,0,340266.072047,53,secondary education,1,civil_partnership,1,F,retiree,0,25378.572,to have a wedding,adulto


[Piensa en los factores de los que suelen depender los ingresos. Eventualmente, tendrás que averiguar si debes usar valores medios o medianos para reemplazar los valores ausentes. Para tomar esta decisión, probablemente querrás ver la distribución de los factores que, según tu análisis, pueden tener un impacto en los ingresos de uno.]

[Crea una tabla que solo tenga datos sin valores ausentes. Estos datos se utilizarán para restaurar los valores ausentes.]

In [51]:
# Crea una tabla sin valores ausentes y muestra algunas de sus filas para asegurarte de que se ve bien
banco_limpio = banco.dropna()
banco_limpio.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,categoria_edad
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,adulto
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,adulto
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,adulto_joven
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,adulto_joven
4,0,340266.072047,53,secondary education,1,civil_partnership,1,F,retiree,0,25378.572,to have a wedding,adulto


In [52]:
# Examina los valores medios de los ingresos en función de los factores que identificaste
banco_limpio.mean()

C:\Users\lucen\AppData\Local\Temp\ipykernel_13116\1532963023.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  banco_limpio.mean()


children                0.479613
days_employed       66914.728907
dob_years              43.452845
education_id            0.819079
family_status_id        0.972249
debt                    0.081184
total_income        26787.568355
dtype: float64

In [53]:
# Examina los valores medianos de los ingresos en función de los factores que identificaste
banco_limpio.median()

C:\Users\lucen\AppData\Local\Temp\ipykernel_13116\697303872.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  banco_limpio.median()


children                0.000000
days_employed        2194.220567
dob_years              42.000000
education_id            1.000000
family_status_id        0.000000
debt                    0.000000
total_income        23202.870000
dtype: float64

Se deben analizar la experiencia laboral en días y el ingreso mensual esperado, con respecto a otros factores, para poder determinar, si estos valores tiene una distribución normal, lineal, o si por el contrarío son dispersos, para luego proceder a rellenar los datos bien sea con la mediana o con la media.

Para ello primeramente debemos analizar la columna de experiencia laboral en día con la categoría de edad, para saber de media, cuantas horas hay distribuidas en cada una, y así, según los rangos de edades de las personas que tiene valores ausentes en lo campos, saber aproximadamente por donde va la distribución. De la misma manera hacer con el ingreso mensual

In [83]:
banco.groupby('categoria_edad')['days_employed'].mean()

categoria_edad
adulto           62902.927952
adulto_joven      2564.997107
tercera_edad    290708.554837
Name: days_employed, dtype: float64

In [88]:
banco.groupby('categoria_edad')['days_employed'].median()

categoria_edad
adulto            2647.493352
adulto_joven      1206.015770
tercera_edad    356191.137667
Name: days_employed, dtype: float64

In [90]:
banco.groupby('categoria_edad')['total_income'].mean()

categoria_edad
adulto          27385.074388
adulto_joven    26836.105974
tercera_edad    23057.777452
Name: total_income, dtype: float64

In [89]:
banco.groupby('categoria_edad')['total_income'].median()

categoria_edad
adulto          23612.936
adulto_joven    23522.246
tercera_edad    19637.056
Name: total_income, dtype: float64

In [87]:
banco.groupby('categoria_edad')['categoria_edad'].count()

categoria_edad
adulto          12744
adulto_joven     6583
tercera_edad     2126
Name: categoria_edad, dtype: int64

In [86]:
banco[banco['days_employed'].isna()].groupby('categoria_edad')['categoria_edad'].count()

categoria_edad
adulto          1261
adulto_joven     632
tercera_edad     209
Name: categoria_edad, dtype: int64

[Repite tales comparaciones para múltiples factores. Asegúrate de considerar diferentes aspectos y explica tu razonamiento.]



[Determina qué características definen mejor los ingresos y decide si utilizarás una mediana o una media. Explica por qué tomaste esta decisión.]


Ya con lo anterior planteado, podemos llegar a conclusiones muy importantes, cuando la media y la mediana se alejan tanto en una distribución como es el caso de la categoría de 'days_employed', es porque los valores se encuentran muy dispersos unos de otros, caso contrario a cuando realizamos la misma operación con 'total_income', conde se evidencia que ambos valores se encuentra muy cercanos en cada categoría de edad. 

Además, a simple vista se evidencia que los valores ausentes son aproximadamente el 10% de la data (como lo calculamos anteriormente) distribuida linealmente para cada categoría de edad, lo que a su vez nos indica que los valores medios y de mediana se deben distribuir de forma lineal para cada categoría, sin ningún inconveniente.

Para confirmar se repetirá el análisis, segmentando por la columna de 'income_type'.

In [91]:
banco.groupby('income_type')['days_employed'].mean()

income_type
business                       2111.524398
civil_servant                  3399.896902
employee                       2326.499216
entrepreneur                    520.848083
paternity_maternity_leave      3296.759962
retiree                      365003.491245
student                         578.751554
unemployed                   366413.652744
Name: days_employed, dtype: float64

In [92]:
banco.groupby('income_type')['days_employed'].median()

income_type
business                       1547.382223
civil_servant                  2689.368353
employee                       1574.202821
entrepreneur                    520.848083
paternity_maternity_leave      3296.759962
retiree                      365213.306266
student                         578.751554
unemployed                   366413.652744
Name: days_employed, dtype: float64

In [93]:
banco.groupby('income_type')['total_income'].mean()


income_type
business                     32386.793835
civil_servant                27343.729582
employee                     25820.841683
entrepreneur                 79866.103000
paternity_maternity_leave     8612.661000
retiree                      21940.394503
student                      15712.260000
unemployed                   21014.360500
Name: total_income, dtype: float64

In [94]:
banco.groupby('income_type')['total_income'].median()

income_type
business                     27577.2720
civil_servant                24071.6695
employee                     22815.1035
entrepreneur                 79866.1030
paternity_maternity_leave     8612.6610
retiree                      18962.3180
student                      15712.2600
unemployed                   21014.3605
Name: total_income, dtype: float64

In [95]:
banco.groupby('income_type')['total_income'].count()

income_type
business                      4577
civil_servant                 1312
employee                     10014
entrepreneur                     1
paternity_maternity_leave        1
retiree                       3443
student                          1
unemployed                       2
Name: total_income, dtype: int64

En este caso la desviación de los datos es menos evidente y se debe a que los valores están subdivididos en más categorías, por lo que las dispersiones cada vez son más lejanas entre sí, por lo que no queda muy claro con que rellenar los valores ausentes siguiendo esta métrica.

Los valores que no varían es porque solamente tenemos una o 2 muestras.

Con ello se puede concluir la mejor métrica de comparación de valores, es la de las categorías de edad, ya que fué donde se evidencio de forma más clara que valores se encuentran dispersos y cuales no.

In [98]:
#  Escribe una función que usaremos para completar los valores ausentes

adul = banco[banco['categoria_edad'] == 'adulto']['total_income'].mean()
adult_jo = banco[banco['categoria_edad'] == 'adulto_joven']['total_income'].mean()
tercera = banco[banco['categoria_edad'] == 'tercera_edad']['total_income'].mean()



        

27385.074388487148


In [56]:
# Comprueba si funciona


In [57]:
# Aplícalo a cada fila


In [58]:
# Comprueba si tenemos algún error


[Si has encontrado errores al preparar los valores para los datos ausentes, probablemente signifique que hay algo especial en los datos de la categoría. Piénsalo un poco: tal vez hará falta arreglar algunas cosas manualmente, si hay suficientes datos para encontrar medianas/medias.]


In [59]:
# Reemplazar los valores ausentes si hay algún error


[Cuando creas que has terminado con `total_income`, comprueba que el número total de valores en esta columna coincida con el número de valores en otras columnas.]

In [60]:
# Comprobar el número de entradas en las columnas



###  Restaurar valores en `days_employed`

[Piensa en los parámetros que pueden ayudarte a restaurar los valores ausentes en esta columna. Eventualmente, tendrás que averiguar si debes usar valores medios o medianos para reemplazar los valores ausentes. Probablemente llevarás a cabo una investigación similar a la que realizaste cuando restauraste los datos en la columna anterior.]

In [61]:
# Distribución de las medianas de `days_employed` en función de los parámetros identificados




In [62]:
# Distribución de las medias de `days_employed` en función de los parámetros identificados

[Decide qué vas a utilizar: medias o medianas. Explica por qué.]

In [63]:
# Escribamos una función que calcule medias o medianas (dependiendo de tu decisión) según el parámetro identificado


In [64]:
# Comprueba que la función funciona



In [65]:
# Aplicar la función al income_type



In [66]:
# Comprueba si la función funcionó



In [67]:
# Reemplazar valores ausentes



[Cuando creas que has terminado con `total_income`, comprueba que el número total de valores en esta columna coincida con el número de valores en otras columnas.]

In [68]:
# Comprueba las entradas en todas las columnas: asegúrate de que hayamos corregido todos los valores ausentes

## Clasificación de datos

[Para poder responder a las preguntas y probar las diferentes hipótesis, querrás trabajar con datos clasificados. Mira las preguntas formuladas que debes responder. Piensa qué parte de los datos tiene que ser clasificada para responder a estas preguntas. A continuación, encontrarás una plantilla a través de la cual puedes trabajar para clasificar los datos. El primer procesamiento paso a paso cubre los datos de texto; el segundo aborda los datos numéricos que necesitan ser clasificados. Puedes usar ambas o ninguna de las instrucciones sugeridas, eso solo depende de ti.]

[Independientemente de cómo decidas abordar la clasificación, asegúrate de proporcionar una explicación clara de la razón por la que tomaste tu decisión. Recuerda: este es tu trabajo y aquí tú tomas todas las decisiones.]


In [69]:
# Muestra los valores de los datos seleccionados para la clasificación



[Vamos a comprobar los valores únicos]

In [70]:
# Comprobar los valores únicos

[¿Qué grupos principales puedes identificar en función de los valores únicos?]

[Según estos temas, probablemente querremos clasificar nuestros datos.]


In [71]:
# Escribamos una función para clasificar los datos en función de temas comunes


In [72]:
# Crea una columna con las categorías y cuenta los valores en ellas



[Si decides clasificar los datos numéricos, también tendrás que crear las categorías para ello.]

In [73]:
# Revisar todos los datos numéricos en la columna seleccionada para la clasificación


In [74]:
# Obtener estadísticas resumidas para la columna



[Decide qué rangos utilizarás para agrupar y explica por qué.]

In [75]:
# Crear una función para clasificar en diferentes grupos numéricos basándose en rangos



In [76]:
# Crear una columna con categorías


In [77]:
# Contar los valores de cada categoría para ver la distribución


## Comprobación de las hipótesis


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [78]:
# Comprueba los datos sobre los hijos y los pagos puntuales


# Calcular la tasa de incumplimiento en función del número de hijos



**Conclusión**

[Escribe tus conclusiones en base a tus manipulaciones y observaciones.]


**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [79]:
# Comprueba los datos del estado familiar y los pagos a tiempo



# Calcular la tasa de incumplimiento basada en el estado familiar



**Conclusión**

[Escribe tus conclusiones en base a tus manipulaciones y observaciones.]

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [80]:
# Comprueba los datos del nivel de ingresos y los pagos a tiempo



# Calcular la tasa de incumplimiento basada en el nivel de ingresos



**Conclusión**

[Escribe tus conclusiones en base a tus manipulaciones y observaciones.]

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [81]:
# Consulta los porcentajes de tasa de incumplimiento para cada propósito del crédito y analízalos



**Conclusión**

[Escribe tus conclusiones en base a tus manipulaciones y observaciones.]


# Conclusión general 

[Enumera tus conclusiones en esta última sección. Asegúrate de incluir todas las conclusiones importantes que hiciste y que te llevaron a la forma en que procesaste y analizaste los datos. Habla de los valores ausentes, los duplicados y las posibles razones y soluciones para los artefactos problemáticos que tuviste que abordar.]

[Enumera tus conclusiones con respecto a las preguntas planteadas aquí también.]
